## Machine Learning: How well does microbial composition predict abduction status?

In [1]:
# Setup
import os
import qiime2 as q2
import pandas as pd

from qiime2 import Visualization

data_dir = '../Alien_data'

In [3]:
# Classify abduction status based on microbial composition

! qiime sample-classifier classify-samples \
  --i-table $data_dir/table-filtered.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --m-metadata-column alleged_abduction \
  --p-test-size 0.2 \
  --p-estimator RandomForestClassifier \
  --p-palette 'enigma' \
  --p-random-state 0 \
  --output-dir $data_dir/small-RF-classifier

Usage: qiime sample-classifier classify-samples [OPTIONS]

  Predicts a categorical sample metadata column using a supervised learning
  classifier. Splits input data into training and test sets. The training
  set is used to train and test the estimator using a stratified k-fold
  cross-validation scheme. This includes optional steps for automated
  feature extraction and hyperparameter optimization. The test set validates
  classification accuracy of the optimized estimator. Outputs classification
  results for test set. For more details on the learning algorithm, see
  http://scikit-learn.org/stable/supervised_learning.html

Inputs:
  --i-table ARTIFACT FeatureTable[Frequency]
                       Feature table containing all features that should be
                       used for target prediction.                  [required]
Parameters:
  --m-metadata-file METADATA
  --m-metadata-column COLUMN  MetadataColumn[Categorical]
                       Categorical metadata column to use

In [4]:
Visualization.load(f'{data_dir}/small-RF-classifier/accuracy_results.qzv')

<visualization: Visualization uuid: e5f3cbc4-c1c2-4254-aa68-7621ef044672>

The overall accuracy is high (87.9%) but not much higher than the baseline accuracy(84.8%). Classifications are clearly skewed towards the "non-abducted" category. Almost all non-abducted samples are correctly classified, but 60% of abducted samples are misclassified as non-abducted. The high baseline accuracy is likely due to the imbalance in abducted vs non-abducted sample sizes, and the correct classification of all non-abducted samples.

In [4]:
# Visualize individual samples' predictions and probabilities

! qiime metadata tabulate \
  --m-input-file $data_dir/small-RF-classifier/test_targets.qza \
  --m-input-file $data_dir/small-RF-classifier/predictions.qza \
  --m-input-file $data_dir/small-RF-classifier/probabilities.qza \
  --o-visualization $data_dir/small-RF-classifier/test_predprob.qzv

Saved Visualization to: ../Alien_data/small-RF-classifier/test_predprob.qzv


In [5]:
Visualization.load(f'{data_dir}/small-RF-classifier/test_predprob.qzv')

<visualization: Visualization uuid: 8daafd69-c08b-4618-8c0e-b693921d4dce>

In [6]:
# Feature importance: which microbial compositions were most important for 
#                     distinguishing abducted vs non-abducted samples?

! qiime metadata tabulate \
    --m-input-file $data_dir/small-RF-classifier/feature_importance.qza \
    --o-visualization $data_dir/small-RF-classifier/feature_importance.qzv

Saved Visualization to: ../Alien_data/small-RF-classifier/feature_importance.qzv


In [7]:
Visualization.load(f'{data_dir}/small-RF-classifier/feature_importance.qzv')

<visualization: Visualization uuid: 7872f614-713d-43ea-874b-3c7a15251721>

In [8]:
! qiime sample-classifier heatmap \
  --i-table $data_dir/table-filtered.qza \
  --i-importance $data_dir/small-RF-classifier/feature_importance.qza \
  --m-sample-metadata-file $data_dir/str_metadata.tsv  \
  --m-sample-metadata-column alleged_abduction \
  --p-group-samples \
  --p-feature-count 20 \
  --o-filtered-table $data_dir/small-RF-classifier/important-feature-table-top-20.qza \
  --o-heatmap $data_dir/small-RF-classifier/important-feature-heatmap.qzv

Saved Visualization to: ../Alien_data/small-RF-classifier/important-feature-heatmap.qzv
Saved FeatureTable[Frequency] to: ../Alien_data/small-RF-classifier/important-feature-table-top-20.qza


In [7]:
Visualization.load(f'{data_dir}/small-RF-classifier/important-feature-heatmap.qzv')

<visualization: Visualization uuid: 11f55cd6-d997-43b4-9c1a-8e27b32ef4cd>

### Repeat with Optimized Feature Selection

(Had no effect on accuracy)

In [10]:
# Classify abduction status based on microbial composition

! qiime sample-classifier classify-samples \
  --i-table $data_dir/table-filtered.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --m-metadata-column alleged_abduction \
  --p-test-size 0.2 \
  --p-estimator RandomForestClassifier \
  --p-optimize-feature-selection \
  --p-palette 'enigma' \
  --p-random-state 0 \
  --output-dir $data_dir/RF-opt-feature-selection

Saved SampleEstimator[Classifier] to: ../Alien_data/RF-opt-feature-selection/sample_estimator.qza
Saved FeatureData[Importance] to: ../Alien_data/RF-opt-feature-selection/feature_importance.qza
Saved SampleData[ClassifierPredictions] to: ../Alien_data/RF-opt-feature-selection/predictions.qza
Saved Visualization to: ../Alien_data/RF-opt-feature-selection/model_summary.qzv
Saved Visualization to: ../Alien_data/RF-opt-feature-selection/accuracy_results.qzv
Saved SampleData[Probabilities] to: ../Alien_data/RF-opt-feature-selection/probabilities.qza
Saved Visualization to: ../Alien_data/RF-opt-feature-selection/heatmap.qzv
Saved SampleData[TrueTargets] to: ../Alien_data/RF-opt-feature-selection/training_targets.qza
Saved SampleData[TrueTargets] to: ../Alien_data/RF-opt-feature-selection/test_targets.qza


In [8]:
Visualization.load(f'{data_dir}/RF-opt-feature-selection/accuracy_results.qzv')

<visualization: Visualization uuid: 55c28585-3187-4f9c-88f8-cfe1f94fd3f4>

### Repeat with Parameter Tuning

Does parameter tuning improve the model's accuracy? No effect.

In [12]:
! qiime sample-classifier classify-samples \
  --i-table $data_dir/table-filtered.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --m-metadata-column alleged_abduction \
  --p-test-size 0.2 \
  --p-optimize-feature-selection \
  --p-parameter-tuning \
  --p-estimator RandomForestClassifier \
  --p-palette 'enigma' \
  --p-random-state 0 \
  --output-dir $data_dir/RF-param-tuning

Saved SampleEstimator[Classifier] to: ../Alien_data/RF-param-tuning/sample_estimator.qza
Saved FeatureData[Importance] to: ../Alien_data/RF-param-tuning/feature_importance.qza
Saved SampleData[ClassifierPredictions] to: ../Alien_data/RF-param-tuning/predictions.qza
Saved Visualization to: ../Alien_data/RF-param-tuning/model_summary.qzv
Saved Visualization to: ../Alien_data/RF-param-tuning/accuracy_results.qzv
Saved SampleData[Probabilities] to: ../Alien_data/RF-param-tuning/probabilities.qza
Saved Visualization to: ../Alien_data/RF-param-tuning/heatmap.qzv
Saved SampleData[TrueTargets] to: ../Alien_data/RF-param-tuning/training_targets.qza
Saved SampleData[TrueTargets] to: ../Alien_data/RF-param-tuning/test_targets.qza


In [9]:
Visualization.load(f'{data_dir}/RF-param-tuning/accuracy_results.qzv')

<visualization: Visualization uuid: c650ee6b-9618-4c57-a8e2-e8b476264936>

### Repeat with More Estimators (more trees)
Does increasing the number of trees improve the model's accuracy?
(Default is 100 trees, here trying 300 and 500) Neither improved overall accuracy.

In [14]:
! qiime sample-classifier classify-samples \
  --i-table $data_dir/table-filtered.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --m-metadata-column alleged_abduction \
  --p-test-size 0.2 \
  --p-estimator RandomForestClassifier \
  --p-optimize-feature-selection \
  --p-parameter-tuning \
  --p-n-estimators 300 \
  --p-palette 'enigma' \
  --p-random-state 0 \
  --output-dir $data_dir/RF-threehundred-trees

Saved SampleEstimator[Classifier] to: ../Alien_data/RF-threehundred-trees/sample_estimator.qza
Saved FeatureData[Importance] to: ../Alien_data/RF-threehundred-trees/feature_importance.qza
Saved SampleData[ClassifierPredictions] to: ../Alien_data/RF-threehundred-trees/predictions.qza
Saved Visualization to: ../Alien_data/RF-threehundred-trees/model_summary.qzv
Saved Visualization to: ../Alien_data/RF-threehundred-trees/accuracy_results.qzv
Saved SampleData[Probabilities] to: ../Alien_data/RF-threehundred-trees/probabilities.qza
Saved Visualization to: ../Alien_data/RF-threehundred-trees/heatmap.qzv
Saved SampleData[TrueTargets] to: ../Alien_data/RF-threehundred-trees/training_targets.qza
Saved SampleData[TrueTargets] to: ../Alien_data/RF-threehundred-trees/test_targets.qza


In [10]:
Visualization.load(f'{data_dir}/RF-threehundred-trees/accuracy_results.qzv')

<visualization: Visualization uuid: 566a4322-693c-4953-8267-85327a02d81c>

In [24]:
! qiime sample-classifier classify-samples \
  --i-table $data_dir/table-filtered.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --m-metadata-column alleged_abduction \
  --p-test-size 0.2 \
  --p-estimator RandomForestClassifier \
  --p-optimize-feature-selection \
  --p-parameter-tuning \
  --p-n-estimators 500 \
  --p-palette 'enigma' \
  --p-random-state 0 \
  --output-dir $data_dir/RF-fivehundred-trees

Saved SampleEstimator[Classifier] to: ../Alien_data/RF-fivehundred-trees/sample_estimator.qza
Saved FeatureData[Importance] to: ../Alien_data/RF-fivehundred-trees/feature_importance.qza
Saved SampleData[ClassifierPredictions] to: ../Alien_data/RF-fivehundred-trees/predictions.qza
Saved Visualization to: ../Alien_data/RF-fivehundred-trees/model_summary.qzv
Saved Visualization to: ../Alien_data/RF-fivehundred-trees/accuracy_results.qzv
Saved SampleData[Probabilities] to: ../Alien_data/RF-fivehundred-trees/probabilities.qza
Saved Visualization to: ../Alien_data/RF-fivehundred-trees/heatmap.qzv
Saved SampleData[TrueTargets] to: ../Alien_data/RF-fivehundred-trees/training_targets.qza
Saved SampleData[TrueTargets] to: ../Alien_data/RF-fivehundred-trees/test_targets.qza


In [11]:
Visualization.load(f'{data_dir}/RF-fivehundred-trees/accuracy_results.qzv')

<visualization: Visualization uuid: cc9bb279-f03d-489a-b36a-547949ade8de>

### Repeat with fewer and more folds (cross-validation)
Default is 5, try 3 and 10. All with 500 trees. Again no effect.

In [18]:
! qiime sample-classifier classify-samples \
  --i-table $data_dir/table-filtered.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --m-metadata-column alleged_abduction \
  --p-test-size 0.2 \
  --p-estimator RandomForestClassifier \
  --p-optimize-feature-selection \
  --p-parameter-tuning \
  --p-n-estimators 500 \
  --p-cv 3 \
  --p-palette 'enigma' \
  --p-random-state 0 \
  --output-dir $data_dir/RF-three-fold

Saved SampleEstimator[Classifier] to: ../Alien_data/RF-three-fold/sample_estimator.qza
Saved FeatureData[Importance] to: ../Alien_data/RF-three-fold/feature_importance.qza
Saved SampleData[ClassifierPredictions] to: ../Alien_data/RF-three-fold/predictions.qza
Saved Visualization to: ../Alien_data/RF-three-fold/model_summary.qzv
Saved Visualization to: ../Alien_data/RF-three-fold/accuracy_results.qzv
Saved SampleData[Probabilities] to: ../Alien_data/RF-three-fold/probabilities.qza
Saved Visualization to: ../Alien_data/RF-three-fold/heatmap.qzv
Saved SampleData[TrueTargets] to: ../Alien_data/RF-three-fold/training_targets.qza
Saved SampleData[TrueTargets] to: ../Alien_data/RF-three-fold/test_targets.qza


In [12]:
Visualization.load(f'{data_dir}/RF-three-fold/accuracy_results.qzv')

<visualization: Visualization uuid: 18dd7d32-9fca-4351-936d-067072dce5e4>

In [20]:
# Removed feature selection parameter because it causes error that 
# "linkage must be computed on at least two observations". Possible that 
# in some partitions of the data, only one feature remains important 
# when using optimized feature selection (all feature importances seem low)

! qiime sample-classifier classify-samples \
  --i-table $data_dir/table-filtered.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --m-metadata-column alleged_abduction \
  --p-test-size 0.2 \
  --p-estimator RandomForestClassifier \
  --p-parameter-tuning \
  --p-n-estimators 500 \
  --p-cv 10 \
  --p-palette 'enigma' \
  --p-random-state 0 \
  --output-dir $data_dir/RF-ten-fold

Saved SampleEstimator[Classifier] to: ../Alien_data/RF-ten-fold/sample_estimator.qza
Saved FeatureData[Importance] to: ../Alien_data/RF-ten-fold/feature_importance.qza
Saved SampleData[ClassifierPredictions] to: ../Alien_data/RF-ten-fold/predictions.qza
Saved Visualization to: ../Alien_data/RF-ten-fold/model_summary.qzv
Saved Visualization to: ../Alien_data/RF-ten-fold/accuracy_results.qzv
Saved SampleData[Probabilities] to: ../Alien_data/RF-ten-fold/probabilities.qza
Saved Visualization to: ../Alien_data/RF-ten-fold/heatmap.qzv
Saved SampleData[TrueTargets] to: ../Alien_data/RF-ten-fold/training_targets.qza
Saved SampleData[TrueTargets] to: ../Alien_data/RF-ten-fold/test_targets.qza


In [13]:
Visualization.load(f'{data_dir}/RF-ten-fold/accuracy_results.qzv')

<visualization: Visualization uuid: ec8669d1-40f5-4251-87db-4e59ee34f36e>

##### Overall, no parameters improved the overall accuracy 

The overall accuracy is almost equal to the baseline accuracy, meaning the model performs basically only as well as a model that simply classifies samples as the most frequent class. This suggests that microbial composition does not provide enough information to predict abduction status.

### Nested cross-validation

In [6]:
# 3-fold, 500 trees
! qiime sample-classifier classify-samples-ncv \
  --i-table $data_dir/table-filtered.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --m-metadata-column alleged_abduction \
  --p-parameter-tuning \
  --p-cv 3 \
  --p-estimator RandomForestClassifier \
  --p-n-estimators 500 \
  --p-random-state 0 \
  --output-dir $data_dir/RF-ncv-classifier-three-fold

Saved SampleData[ClassifierPredictions] to: ../Alien_data/RF-ncv-classifier-three-fold/predictions.qza
Saved FeatureData[Importance] to: ../Alien_data/RF-ncv-classifier-three-fold/feature_importance.qza
Saved SampleData[Probabilities] to: ../Alien_data/RF-ncv-classifier-three-fold/probabilities.qza


In [7]:
# Classify abduction status based on microbial composition

! qiime sample-classifier confusion-matrix \
  --i-predictions $data_dir/RF-ncv-classifier-three-fold/predictions.qza \
  --i-probabilities $data_dir/RF-ncv-classifier-three-fold/probabilities.qza \
  --m-truth-file $data_dir/str_metadata.tsv \
  --m-truth-column alleged_abduction \
  --o-visualization $data_dir/RF-ncv-classifier-three-fold/ncv_confusion_matrix.qzv

Saved Visualization to: ../Alien_data/RF-ncv-classifier-three-fold/ncv_confusion_matrix.qzv


In [8]:
Visualization.load(f'{data_dir}/RF-ncv-classifier-three-fold/ncv_confusion_matrix.qzv')

<visualization: Visualization uuid: d1a2cdea-4414-41aa-a206-e30d66ffa650>

In [9]:
# 5-fold, 500 trees
! qiime sample-classifier classify-samples-ncv \
  --i-table $data_dir/table-filtered.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --m-metadata-column alleged_abduction \
  --p-parameter-tuning \
  --p-cv 5 \
  --p-estimator RandomForestClassifier \
  --p-n-estimators 500 \
  --p-random-state 0 \
  --output-dir $data_dir/RF-ncv-classifier-five-fold

Saved SampleData[ClassifierPredictions] to: ../Alien_data/RF-ncv-classifier-five-fold/predictions.qza
Saved FeatureData[Importance] to: ../Alien_data/RF-ncv-classifier-five-fold/feature_importance.qza
Saved SampleData[Probabilities] to: ../Alien_data/RF-ncv-classifier-five-fold/probabilities.qza


In [10]:
# Classify abduction status based on microbial composition
! qiime sample-classifier confusion-matrix \
  --i-predictions $data_dir/RF-ncv-classifier-five-fold/predictions.qza \
  --i-probabilities $data_dir/RF-ncv-classifier-five-fold/probabilities.qza \
  --m-truth-file $data_dir/str_metadata.tsv \
  --m-truth-column alleged_abduction \
  --o-visualization $data_dir/RF-ncv-classifier-five-fold/ncv_confusion_matrix.qzv

Saved Visualization to: ../Alien_data/RF-ncv-classifier-five-fold/ncv_confusion_matrix.qzv


In [11]:
Visualization.load(f'{data_dir}/RF-ncv-classifier-five-fold/ncv_confusion_matrix.qzv')

<visualization: Visualization uuid: 488b7c88-549f-45a3-ac52-d470b37ddbc7>

In [3]:
# 10-fold, 500 trees
! qiime sample-classifier classify-samples-ncv \
  --i-table $data_dir/table-filtered.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --m-metadata-column alleged_abduction \
  --p-parameter-tuning \
  --p-cv 10 \
  --p-estimator RandomForestClassifier \
  --p-n-estimators 500 \
  --p-random-state 0 \
  --output-dir $data_dir/RF-ncv-classifier-ten-fold

^C

Aborted!


In [15]:
# Classify abduction status based on microbial composition

! qiime sample-classifier confusion-matrix \
  --i-predictions $data_dir/RF-ncv-classifier-ten-fold/predictions.qza \
  --i-probabilities $data_dir/RF-ncv-classifier-ten-fold/probabilities.qza \
  --m-truth-file $data_dir/str_metadata.tsv \
  --m-truth-column alleged_abduction \
  --o-visualization $data_dir/RF-ncv-classifier-ten-fold/ncv_confusion_matrix.qzv

Saved Visualization to: ../Alien_data/RF-ncv-classifier-three-fold/ncv_confusion_matrix.qzv


In [ ]:
Visualization.load(f'{data_dir}/RF-ncv-classifier-ten-fold/ncv_confusion_matrix.qzv')

### Repeat with Balanced Sample Sizes

Limitation of the method below: cannot set seed for reproducible random sampling of the non-abducted samples.

In [16]:
# Randomly sample non-abducted samples in metadata
from random import sample

meta = pd.read_csv(f"{data_dir}/str_metadata.tsv", sep = "\t")
meta.head()

,sampleid,stool_consistency,hct_source,disease,categorical_time_relative_to_engraftment,week_relative_to_hct,timepoint_of_transplant,day_relative_to_nearest_hct,alleged_abduction
0,N4VICF,formed,cord,Myelodysplastic Syndromes,pre,one week before HCT,6.0,-6.0,non_abducted
1,8A0F9A,formed,cord,Leukemia,pre,two weeks before HCT,7.0,-7.0,non_abducted
2,5Y49IM,semi-formed,cord,Leukemia,peri,one week before HCT,7.0,0.0,abducted
3,ZKJI45,semi-formed,cord,Leukemia,post,one week after HCT,7.0,8.0,non_abducted
4,2I7SIQ,liquid,cord,Leukemia,peri,one week before HCT,0.0,0.0,abducted


In [17]:
meta_nonab = meta.loc[meta['alleged_abduction'] == "non_abducted"]
meta_nonab.head()

,sampleid,stool_consistency,hct_source,disease,categorical_time_relative_to_engraftment,week_relative_to_hct,timepoint_of_transplant,day_relative_to_nearest_hct,alleged_abduction
0,N4VICF,formed,cord,Myelodysplastic Syndromes,pre,one week before HCT,6.0,-6.0,non_abducted
1,8A0F9A,formed,cord,Leukemia,pre,two weeks before HCT,7.0,-7.0,non_abducted
3,ZKJI45,semi-formed,cord,Leukemia,post,one week after HCT,7.0,8.0,non_abducted
6,XO59R8,liquid,cord,Leukemia,pre,one week before HCT,1.0,-1.0,non_abducted
7,AFG7YZ,semi-formed,cord,Leukemia,post,two weeks before HCT,1.0,15.0,non_abducted


In [18]:
# Get number of non-abducted samples that need to be REMOVED 
ncut = len(meta_nonab) - len(meta.loc[meta['alleged_abduction'] == "abducted"])
ncut

109

In [19]:
cutIDs = sample(list(meta_nonab["sampleid"]), ncut)
cutIDs[:10]

['NV2MHM',
 '63BB0Q',
 'TEHZN5',
 '3CGEF6',
 '0KB68F',
 '3DFM85',
 'LIFJEK',
 '82O5JA',
 '1IMOSV',
 'P0A2X1']

In [20]:
meta.drop(axis = 0, index = meta.loc[meta["sampleid"].isin(cutIDs)].index,  inplace=True, errors='raise')

In [21]:
len(meta.loc[meta["alleged_abduction"] == "non_abducted"])

26

In [22]:
# write new metadata file
meta.to_csv(f"{data_dir}/meta_balanced.tsv", sep = "\t", index=False)

In [23]:
# Double-check the new metadata file
meta = pd.read_csv(f"{data_dir}/meta_balanced.tsv", sep = "\t")
meta.head()

,sampleid,stool_consistency,hct_source,disease,categorical_time_relative_to_engraftment,week_relative_to_hct,timepoint_of_transplant,day_relative_to_nearest_hct,alleged_abduction
0,5Y49IM,semi-formed,cord,Leukemia,peri,one week before HCT,7.0,0.0,abducted
1,ZKJI45,semi-formed,cord,Leukemia,post,one week after HCT,7.0,8.0,non_abducted
2,2I7SIQ,liquid,cord,Leukemia,peri,one week before HCT,0.0,0.0,abducted
3,PCUMU7,semi-formed,cord,Leukemia,post,two weeks before HCT,0.0,16.0,abducted
4,XO59R8,liquid,cord,Leukemia,pre,one week before HCT,1.0,-1.0,non_abducted


In [25]:
# Filter feature table to only include the remaining samples 

! qiime feature-table filter-samples \
  --i-table $data_dir/table-filtered.qza \
  --m-metadata-file $data_dir/meta_balanced.tsv \
  --o-filtered-table $data_dir/table-filtered-balanced.qza 

Saved FeatureTable[Frequency] to: ../Alien_data/table-filtered-balanced.qza


In [26]:
# Rerun nested cross-validation

# 3-fold, 100 trees
! qiime sample-classifier classify-samples-ncv \
  --i-table $data_dir/table-filtered-balanced.qza \
  --m-metadata-file $data_dir/meta_balanced.tsv \
  --m-metadata-column alleged_abduction \
  --p-parameter-tuning \
  --p-cv 3 \
  --p-estimator RandomForestClassifier \
  --p-n-estimators 100 \
  --p-random-state 0 \
  --output-dir $data_dir/balanced-ncv-three-fold

Saved SampleData[ClassifierPredictions] to: ../Alien_data/balanced-ncv-three-fold/predictions.qza
Saved FeatureData[Importance] to: ../Alien_data/balanced-ncv-three-fold/feature_importance.qza
Saved SampleData[Probabilities] to: ../Alien_data/balanced-ncv-three-fold/probabilities.qza


In [27]:
# Classify abduction status based on microbial composition

! qiime sample-classifier confusion-matrix \
  --i-predictions $data_dir/balanced-ncv-three-fold/predictions.qza \
  --i-probabilities $data_dir/balanced-ncv-three-fold/probabilities.qza \
  --m-truth-file $data_dir/meta_balanced.tsv \
  --m-truth-column alleged_abduction \
  --o-visualization $data_dir/balanced-ncv-three-fold/ncv_confusion_matrix.qzv

Saved Visualization to: ../Alien_data/balanced-ncv-three-fold/ncv_confusion_matrix.qzv


In [2]:
Visualization.load(f'{data_dir}/balanced-ncv-three-fold/ncv_confusion_matrix.qzv')

<visualization: Visualization uuid: 3045d1da-99e6-460c-a5ae-37f05497f3e3>

Now the baseline accuracy is lower because the frequency of the classes are equal, and the overall accuracy is even slightly worse than random chance. Repeating with more trees and more folds:

In [29]:
# 5-fold, 300 trees

! qiime sample-classifier classify-samples-ncv \
  --i-table $data_dir/table-filtered-balanced.qza \
  --m-metadata-file $data_dir/meta_balanced.tsv \
  --m-metadata-column alleged_abduction \
  --p-parameter-tuning \
  --p-cv 5 \
  --p-estimator RandomForestClassifier \
  --p-n-estimators 300 \
  --p-random-state 0 \
  --output-dir $data_dir/balanced-ncv-five-fold

Saved SampleData[ClassifierPredictions] to: ../Alien_data/balanced-ncv-five-fold/predictions.qza
Saved FeatureData[Importance] to: ../Alien_data/balanced-ncv-five-fold/feature_importance.qza
Saved SampleData[Probabilities] to: ../Alien_data/balanced-ncv-five-fold/probabilities.qza


In [4]:
# Classify abduction status based on microbial composition

! qiime sample-classifier confusion-matrix \
  --i-predictions $data_dir/balanced-ncv-five-fold/predictions.qza \
  --i-probabilities $data_dir/balanced-ncv-five-fold/probabilities.qza \
  --m-truth-file $data_dir/meta_balanced.tsv \
  --m-truth-column alleged_abduction \
  --o-visualization $data_dir/balanced-ncv-five-fold/ncv_confusion_matrix.qzv

Saved Visualization to: ../Alien_data/balanced-ncv-five-fold/ncv_confusion_matrix.qzv


In [5]:
Visualization.load(f'{data_dir}/balanced-ncv-five-fold/ncv_confusion_matrix.qzv')

<visualization: Visualization uuid: 8249b7b0-cd45-436b-a7ca-f5a6b3fb27c6>

### Trying Linear Support Vector Classifier (SVC)

With 5-fold nested cross-validation (turns out even worse)

In [5]:
! qiime sample-classifier classify-samples-ncv \
  --i-table $data_dir/table-filtered.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --m-metadata-column alleged_abduction \
  --p-parameter-tuning \
  --p-cv 5 \
  --p-estimator LinearSVC \
  --p-random-state 0 \
  --output-dir $data_dir/LSVC-ncv-five-fold

Saved SampleData[ClassifierPredictions] to: ../Alien_data/LSVC-ncv-five-fold/predictions.qza
Saved FeatureData[Importance] to: ../Alien_data/LSVC-ncv-five-fold/feature_importance.qza
Saved SampleData[Probabilities] to: ../Alien_data/LSVC-ncv-five-fold/probabilities.qza


In [6]:
# Classify abduction status based on microbial composition

! qiime sample-classifier confusion-matrix \
  --i-predictions $data_dir/LSVC-ncv-five-fold/predictions.qza \
  --i-probabilities $data_dir/LSVC-ncv-five-fold/probabilities.qza \
  --m-truth-file $data_dir/str_metadata.tsv \
  --m-truth-column alleged_abduction \
  --o-visualization $data_dir/LSVC-ncv-five-fold/LSVC_ncv_confusion_matrix.qzv

Saved Visualization to: ../Alien_data/LSVC-ncv-five-fold/LSVC_ncv_confusion_matrix.qzv


In [7]:
Visualization.load(f'{data_dir}/LSVC-ncv-five-fold/LSVC_ncv_confusion_matrix.qzv')

<visualization: Visualization uuid: ecab5167-ac1a-478e-9d56-59f576ef2808>